In [1]:
v = [3,8,1001,8,10,8,105,1,0,0,21,34,51,64,73,98,179,260,341,422,99999,3,9,102,4,9,9,1001,9,4,9,4,9,99,3,9,1001,9,4,9,1002,9,3,9,1001,9,5,9,4,9,99,3,9,101,5,9,9,102,5,9,9,4,9,99,3,9,101,5,9,9,4,9,99,3,9,1002,9,5,9,1001,9,3,9,102,2,9,9,101,5,9,9,1002,9,2,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,99]



In [2]:
import numpy as np
def modes(code):
    inm = np.full(4, 0)
    if len(code) > 2:
        for x in range(len(code) - 2, 0, -1):
            if code[x - 1] == "1":
                inm[len(code) - x - 2 + 1] = 1

    return inm

In [3]:
def parameters1(v, p, m):
    p1 = v[p+1]
    if m[1] == 0:
        p1 = v[p1]

    return p1

def parameters2(v, p, m):
    p1 = v[p+1]
    if m[1] == 0:
        p1 = v[p1]

    p2 = v[p+2]
    if m[2] == 0:
        p2 = v[p2]
        
    return [p1, p2]

In [4]:
class Amp:
    p = 0
    #out = inputValues[len(inputValues) - 1]
    inputCounter = 0


    def __init__(self, v, inputValues):
        self.v = v
        self.inputValues = inputValues
        self.inputCounter = 0
        self.p = 0
        self.halt = False
    
    def run(self, inputValue):
        
        #print(self.v)
        
        #print(">", inputValue)
        if self.halt:
            print("H", self.inputValues[0])
        
        self.inputValues.append(inputValue)
        
        while(self.v[self.p] != 99):

            code = str(self.v[self.p])
            instruction = int(code[len(code) - 2:])
            m = modes(code)

            if instruction == 1: # add
                p1, p2 = parameters2(self.v, self.p, m)

                ### Parameters that an instruction writes to will never be in immediate mode.   
                self.v[self.v[self.p+3]] = p1 + p2

                self.p += 4

            elif instruction == 2: # mul
                p1, p2 = parameters2(self.v, self.p, m)

                ### Parameters that an instruction writes to will never be in immediate mode.    
                self.v[self.v[self.p+3]] = p1 * p2

                self.p += 4

            elif instruction == 3: # input
                #if self.inputCounter < len(self.inputValues):
                iv = self.inputValues[self.inputCounter]

                #else:
                #    iv = self.inputValues[len(self.inputValues) - 1]

                self.inputCounter += 1
                #print("<", iv, self.inputCounter, self.p)
                self.v[self.v[self.p+1]] = iv    

                self.p += 2

            elif instruction == 4: # output  
                iv = parameters1(self.v, self.p, m)
                #print(">", iv, self.inputValues[0], self.p, code)
                #print("V", self.v)
                self.out = iv #.append(iv)
                self.p += 2

                return self.out, False


            elif instruction == 5: # jump if true
                p1, p2 = parameters2(self.v, self.p, m)

                if p1 != 0:
                    self.p = p2
                else:
                    self.p += 3

            elif instruction == 6: # jump if false
                p1, p2 = parameters2(self.v, self.p, m)

                if p1 == 0:
                    self.p = p2
                else:
                    p += 3

            elif instruction == 7: # less than
                p1, p2 = parameters2(self.v, self.p, m)

                if p1 < p2:
                    self.v[self.v[self.p+3]] = 1
                else:
                    self.v[self.v[self.p+3]] = 0

                self.p += 4

            elif instruction == 8: # equal
                p1, p2 = parameters2(self.v, self.p, m)

                if p1 == p2:
                    self.v[self.v[self.p+3]] = 1
                else:
                    self.v[self.v[self.p+3]] = 0

                self.p += 4

            else:
                print("error", self.p)
        self.halt = True
        #print("*", self.out, self.inputValues[0])
        return [self.out, True]



In [5]:
maxout = 0

v_test = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]


import numpy as np



for i5 in range(5, 10):
    for i4 in range(5, 10):
        for i3 in range(5, 10):
            for i2 in range(5, 10):
                for i1 in range(5, 10):
                    
                    if not (i1 == i2 or i1 == i3 or i1 == i4 or i1 == i5 or i2 == i3 or i2 == i4 or i2 == i5 or i3 == i4 or i3 == i5 or i4 == i5):


                        amp1 = Amp(np.array(v).copy(), [i1])
                        amp2 = Amp(np.array(v).copy(), [i2])
                        amp3 = Amp(np.array(v).copy(), [i3])
                        amp4 = Amp(np.array(v).copy(), [i4])
                        amp5 = Amp(np.array(v).copy(), [i5])

                        out = 0
                        done = False
                        while not done:
                            out, done1 = amp1.run(out)
                            out, done2 = amp2.run(out)
                            out, done3 = amp3.run(out)
                            out, done4 = amp4.run(out)
                            out, done5 = amp5.run(out)

                            #print("********    ", done1, done2, done3, done4, done5)

                            done = done5

                        if out > maxout:
                            maxout = out


#assert maxout == 14902



In [6]:
print("MAXOUT", maxout)

MAXOUT 6489132
